In [ ]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from scipy.stats import spearmanr, pearsonr

In [ ]:
# Setting parameters
recordings = 26
dir = '/Users/lillianwang/Documents/bird-counts-25/'
birdnet_dir = dir+'data/birdnet/'
ebird_dir = dir+'data/ebird/'

In [ ]:
def load_data(dir):
    files = sorted([f for f in os.listdir(dir) if f.endswith('.csv')])
    dfs = []
    for i, file in enumerate(files):
        df = pd.read_csv(dir+file, encoding="latin1")
        df['Recording'] = i+1
        dfs.append(df)
    return dfs

In [ ]:
# Read data
birdnet_dfs = load_data(birdnet_dir)
ebird_dfs = load_data(ebird_dir)

# Strip durations to digits
def extract_numbers(text):
    return ''.join(filter(str.isdigit, text))
    
for ebird_df, birdnet_df in zip(ebird_dfs, birdnet_dfs):
    if not ebird_df.empty:
        birdnet_df['Duration'] = ebird_df['Duration'].apply(extract_numbers)
        ebird_df['Duration'] = ebird_df['Duration'].apply(extract_numbers)

combined_birdnet = pd.concat(birdnet_dfs)
combined_ebird = pd.concat(ebird_dfs)

In [ ]:
# Correcting inconsistent common names
name_corrections = {
    'Eastern Towhee (Red-eyed)': 'Eastern Towhee',
    'Song Sparrow (melodia/atlantica)': 'Song Sparrow',
    'Yellow Warbler (Northern)': 'Yellow Warbler',
    'Northern House Wren (Northern)': 'House Wren',
    'Red-winged Blackbird (Red-winged)': 'Red-winged Blackbird',
    'Common Grackle (Bronzed)': 'Common Grackle',
    'Hairy Woodpecker (Eastern)': 'Hairy Woodpecker',
    'Eastern Meadowlark (Eastern)': 'Eastern Meadowlark',
    'Downy Woodpecker (Eastern)': 'Downy Woodpecker',
    'Northern Cardinal (Northern)': 'Northern Cardinal',
    'White-breasted Nuthatch (Eastern)': 'White-breasted Nuthatch',
    'Northern Flicker (Yellow-shafted)': 'Northern Flicker'
}

for df in ebird_dfs:
  df['Species'] = df['Species'].replace(name_corrections)

combined_ebird['Species'] = combined_ebird['Species'].replace(name_corrections)

## Filters

In [ ]:
def filter_data(df):
  data_points = df['Species'].value_counts()
  filtered_data = df[df['Species'].isin(data_points[data_points > 2].index)]
  return filtered_data

In [ ]:
def remove_outliers(df):
    def filter_group(group):
        q_low = group['Detection rate'].quantile(0.025)
        q_high = group['Detection rate'].quantile(0.975)
        return group[(group['Detection rate'] > q_low) & (group['Detection rate'] < q_high)]

    return df.groupby('Species').apply(filter_group)

In [ ]:
# Find species with at least one detection and entry in the same recording
true_pos_list = []

for i in range(recordings):
  birdnet_species = set(birdnet_dfs[i]['Common name'])
  ebird_species = set(ebird_dfs[i]['Species'])
  true_pos_list.append(birdnet_species & ebird_species)

true_pos_list_flat = [item for sublist in true_pos_list for item in sublist]
true_pos = list(set(true_pos_list_flat))

## Detection rates

In [ ]:
# Detections vs. count by recording
plt.figure(figsize=(8, 6))

total_detections = []
total_count = []

for i in range(recordings):
    detections_count = len(birdnet_dfs[i])
    duration = int(ebird_dfs[i]['Duration'].drop_duplicates().sum())
    total_detections.append(detections_count/duration)

for i in range(recordings):
    count = ebird_dfs[i]['Count'].sum()
    total_count.append(count)

plt.plot(range(recordings), total_count, label='Count', color='b')
plt.plot(range(recordings), total_detections, label='Detections', color='r')

plt.title('Detections vs. count')
plt.xlabel('Recording')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('detections-vs-count.png', dpi=200)
plt.show()


In [ ]:
# Detections
plt.figure(figsize=(8, 6))

x_vals = []
y_vals = []
labels = []

for species in true_pos:
  detections_rows = combined_birdnet[combined_birdnet['Common name'] == species]
  total_detections = len(detections_rows)
  individuals_rows = combined_ebird[combined_ebird['Species'] == species]
  total_count = individuals_rows['Count'].sum()

  x_vals.append(total_detections)
  y_vals.append(total_count)
  labels.append(species)

plt.scatter(x_vals, y_vals, color='b')

texts = []
for i, label in enumerate(labels):
    texts.append(plt.text(x_vals[i], y_vals[i], label, fontsize=8))
adjust_text(texts, arrowprops=dict(arrowstyle='-', lw=0.5), force_text=1)

plt.title('Detection rate by species')
plt.xlabel('Detections')
plt.ylabel('Count')
plt.grid(True)

plt.tight_layout()
plt.savefig('detections-all-species.png', dpi=200)
plt.show()


In [ ]:
# Detection rates (normalized by min)
plt.figure(figsize=(8, 6))

x_vals = []
y_vals = []
labels = []

for species in true_pos:
  detections_rows = combined_birdnet[combined_birdnet['Common name'] == species]
  total_detections = len(detections_rows)
  present_recordings = set(detections_rows['Recording'])
  total_duration = 0

  for recording in present_recordings:
     var = combined_ebird[combined_ebird['Recording'] == recording]
     total_duration += int(var['Duration'].drop_duplicates().sum())

  detection_rate = total_detections/total_duration

  individuals_rows = combined_ebird[combined_ebird['Species'] == species]
  total_count = individuals_rows['Count'].sum()

  x_vals.append(detection_rate)
  y_vals.append(total_count)
  labels.append(species)

plt.scatter(x_vals, y_vals, color='b')

texts = []
for i, label in enumerate(labels):
    texts.append(plt.text(x_vals[i], y_vals[i], label, fontsize=8))
adjust_text(texts, arrowprops=dict(arrowstyle='-', lw=0.5), force_text=1)

plt.title('Detection rate by species (normalized by min)')
plt.xlabel('Detections/min')
plt.ylabel('Count')
plt.grid(True)

plt.tight_layout()
plt.savefig('detection-rate-all-species.png', dpi=200)
plt.show()


In [ ]:
# Scatterplots and regressions of detection rates by species
fig, axes = plt.subplots(nrows=len(true_pos), ncols=1, figsize=(8, len(true_pos)*2.5))

for i, species in enumerate(true_pos):
  x_vals = []
  y_vals = []
  labels = []
  colors = []

  detections_rows = combined_birdnet[combined_birdnet['Common name'] == species]
  present_recordings = set(detections_rows['Recording'])

  for recording in present_recordings:
    ebird_rows = combined_ebird[combined_ebird['Recording'] == recording]
    count = ebird_rows[ebird_rows['Species'] == species]['Count'].sum()

    birdnet_rows = combined_birdnet[combined_birdnet['Recording'] == recording]
    num_detections = len(birdnet_rows[birdnet_rows['Common name'] == species])
    recording_length = int(ebird_rows['Duration'].drop_duplicates().sum())

    detection_rate = num_detections/recording_length
    total_confidence = birdnet_rows[birdnet_rows['Common name'] == species]['Confidence'].sum()

    x_vals.append(detection_rate)
    y_vals.append(count)
    labels.append(recording)
    colors.append(total_confidence)

  axes[i].scatter(x_vals, y_vals, c=colors, cmap='viridis')
  for x, y, label in zip(x_vals, y_vals, labels):
    axes[i].annotate(label, (x,y), textcoords='offset points', xytext=(5,5))

  axes[i].set_title('BirdNET detections/min vs count: '+species)
  axes[i].set_xlabel('Detections/min')
  axes[i].set_ylabel('Count')
  axes[i].set_xlim(0, max(x_vals)+0.5)
  axes[i].set_ylim(0, max(y_vals)+0.5)

  # Plot regression lines
  # b, a = np.polyfit(x_vals, y_vals, deg=1)
  # xseq = np.linspace(0, 10, num=100)
  # axes[i].plot(xseq, a + b * xseq, lw=0.5)

  # # b, a = np.polyfit(np.log(x_vals), y_vals, deg=1)
  # # xseq = np.linspace(0, 10, num=100)
  # xseq_log = np.log(xseq)
  # axes[i].plot(xseq, a + b * xseq_log, lw=0.5)

plt.tight_layout()
plt.savefig('detection-rate-by-species.png', dpi=200)
plt.show()


In [ ]:
# Scatterplots and regressions of confidence rates by species
fig, axes = plt.subplots(nrows=len(true_pos), ncols=1, figsize=(8, len(true_pos)*2.5))

for i, species in enumerate(true_pos):
  x_vals = []
  y_vals = []
  labels = []

  detections_rows = combined_birdnet[combined_birdnet['Common name'] == species]
  present_recordings = set(detections_rows['Recording'])

  for recording in present_recordings:
    ebird_rows = combined_ebird[combined_ebird['Recording'] == recording]
    count = ebird_rows[ebird_rows['Species'] == species]['Count'].sum()

    birdnet_rows = combined_birdnet[combined_birdnet['Recording'] == recording]
    total_confidence = birdnet_rows[birdnet_rows['Common name'] == species]['Confidence'].sum()

    x_vals.append(total_confidence)
    y_vals.append(count)
    labels.append(recording)

  axes[i].scatter(x_vals, y_vals, color='skyblue')
  for x, y, label in zip(x_vals, y_vals, labels):
    axes[i].annotate(label, (x, y), xytext=(5, 5), textcoords='offset points', ha='left')

  axes[i].set_title('Total confidence vs count: '+species)
  axes[i].set_xlabel('Total confidence')
  axes[i].set_ylabel('Count')
  axes[i].set_xlim(0, max(x_vals)*1.25)
  axes[i].set_ylim(0, max(y_vals)*1.25)

  # Plot regression lines
  # b, a = np.polyfit(x_vals, y_vals, deg=1)
  # xseq = np.linspace(0, 10, num=100)
  # axes[i].plot(xseq, a + b * xseq, lw=0.5)

  # b, a = np.polyfit(np.log(x_vals), y_vals, deg=1)
  # xseq = np.linspace(0, 10, num=100)
  # xseq_log = np.log(xseq)
  # axes[i].plot(xseq, a + b * xseq_log, lw=0.5)

plt.tight_layout()
plt.savefig('confidence-rate-by-species.png', dpi=200)
plt.show()


In [ ]:
# Detection rate boxplots
fig, axes = plt.subplots(nrows=len(true_pos), ncols=1, figsize=(8, len(true_pos)*2.5))

for i, species in enumerate(true_pos):
 detections_rows = combined_birdnet[combined_birdnet['Common name'] == species]
 present_recordings = set(detections_rows['Recording'])

 detection_rates = []
 total_detections = 0
 total_count = 0

 for recording in present_recordings:
   ebird_rows = combined_ebird[combined_ebird['Recording'] == recording]
   count = ebird_rows[ebird_rows['Species'] == species]['Count'].sum()

   birdnet_rows = combined_birdnet[combined_birdnet['Recording'] == recording]
   num_detections = len(birdnet_rows[birdnet_rows['Common name'] == species])

   recording_length = int(ebird_rows['Duration'].drop_duplicates().sum())
   if count == 0:
    continue
   detection_rate = num_detections/(recording_length * count)

   detection_rates.append(detection_rate)
   total_detections += num_detections
   total_count += count

 sns.boxplot(ax=axes[i], x=detection_rates, color='skyblue')
 ci = 1.96*np.std(detection_rates)/np.sqrt(len(detection_rates))
 axes[i].errorbar(np.mean(detection_rates), 0, c='b', xerr=ci, fmt='o', capsize=5)
 axes[i].text(np.mean(detection_rates)+ci, 0, str(np.round(np.mean(detection_rates)+ci, 3)))
 axes[i].text(np.mean(detection_rates)-ci, 0, str(np.round(np.mean(detection_rates)-ci, 3)))
 axes[i].tick_params(rotation=45, labelsize=7.5)

 axes[i].set_title(f'{species} detection rate\n(Detections: {total_detections}, Count: {total_count})')
 axes[i].set_xlabel('Detection rate')

plt.tight_layout()
plt.savefig('detection-rate-boxplots.png', dpi=200)
plt.show()

In [ ]:
# Detection rates swarm plots
plt.figure(figsize=(8, len(true_pos)*0.5))

swarm_data = []

for i, species in enumerate(true_pos):
 detections_rows = combined_birdnet[combined_birdnet['Common name'] == species]
 present_recordings = set(detections_rows['Recording'])

 for recording in present_recordings:
   ebird_rows = combined_ebird[combined_ebird['Recording'] == recording]
   count = ebird_rows[ebird_rows['Species'] == species]['Count'].sum()

   birdnet_rows = combined_birdnet[combined_birdnet['Recording'] == recording]
   num_detections = len(birdnet_rows[birdnet_rows['Common name'] == species])

   recording_length = int(ebird_rows['Duration'].drop_duplicates().sum())
   if count == 0:
    continue
   detection_rate = num_detections/(recording_length * count)

   swarm_data.append([species, detection_rate])

swarm_df = pd.DataFrame(swarm_data, columns=['Species', 'Detection rate'])
sns.swarmplot(data=swarm_df, x='Detection rate', y='Species', color='b', size=3)

plt.title('Species detection rates (detections/individual/min)')
plt.xlim(0, max(swarm_df['Detection rate'])*1.05)

plt.tight_layout()
plt.savefig('detection-rate-swarm.png', dpi=200)
plt.show()

In [ ]:
# Detection rates violin plots
plt.figure(figsize=(8, len(true_pos)*0.5))

violin_data = []

for i, species in enumerate(true_pos):
 detections_rows = combined_birdnet[combined_birdnet['Common name'] == species]
 present_recordings = set(detections_rows['Recording'])

 for recording in present_recordings:
   ebird_rows = combined_ebird[combined_ebird['Recording'] == recording]
   count = ebird_rows[ebird_rows['Species'] == species]['Count'].sum()

   birdnet_rows = combined_birdnet[combined_birdnet['Recording'] == recording]
   num_detections = len(birdnet_rows[birdnet_rows['Common name'] == species])

   recording_length = int(ebird_rows['Duration'].drop_duplicates().sum())
   if count == 0:
    continue
   detection_rate = num_detections/(recording_length * count)

   violin_data.append([species, detection_rate])

violin_df = pd.DataFrame(violin_data, columns=['Species', 'Detection rate'])
violin_df = remove_outliers(violin_df)
violin_df = filter_data(violin_df)
sns.violinplot(data=violin_df, x='Detection rate', y='Species', color='skyblue')
sns.swarmplot(data=violin_df, x='Detection rate', y='Species', color='b', size=3)

plt.title('Species detection rates (detections/individual/min)')
plt.xlim(0, max(violin_df['Detection rate'])*1.05)

plt.tight_layout()
plt.savefig('detection-rate-violin.png', dpi=200)
plt.show()

## Statistical analysis

In [ ]:
# Pearson and Spearman metrics
correlation_data = []

for species in true_pos:
    detections_rows = combined_birdnet[combined_birdnet['Common name'] == species]
    present_recordings = set(detections_rows['Recording'])

    for recording in present_recordings:
        ebird_rows = combined_ebird[combined_ebird['Recording'] == recording]
        count = ebird_rows[ebird_rows['Species'] == species]['Count'].sum()

        birdnet_rows = combined_birdnet[combined_birdnet['Recording'] == recording]
        num_detections = len(birdnet_rows[birdnet_rows['Common name'] == species])

        if count == 0:
            continue
        detection_rate = num_detections/count

        correlation_data.append([species, count, detection_rate])

# Find correlation metrics
correlation_df = pd.DataFrame(correlation_data, columns=['Species', 'Count', 'Detection rate'])
corr_pearson, p_pearson = pearsonr(correlation_df['Count'], correlation_df['Detection rate'])
corr_spearman, p_spearman = spearmanr(correlation_df['Count'], correlation_df['Detection rate'])

print(f'Pearson: r={corr_pearson:.3f}, p={p_pearson:.3f}')
print(f'Spearman: r={corr_spearman:.3f}, p={p_spearman:.3f}')